In [1]:
"""
use this script to grab false classified samples
"""

'\nuse this script to grab false classified samples\n'

In [1]:
# libs import
import csv
import pandas as pd
import shutil
import os
import numpy as np
import math
import glob

In [8]:
# parameter setting
dir_src = '/home/gtx980/Desktop/CCP/Results/negative_samples/0714_5fold//'

fp_crit = 0.99
fn_crit = 0.1

##
dir_fp = dir_src + 'fps_' + str(fp_crit) + '/'
dir_fn = dir_src + 'fns_' + str(fn_crit) + '/'
vis_merge = dir_src + 'merge/'

###
# create dst dir if not exist
if not os.path.exists(dir_fp):
    os.makedirs(dir_fp)
    
if not os.path.exists(dir_fn):
    os.makedirs(dir_fn)
    
    
print(os.listdir(dir_src))

['fns_0.1', 'testing_model_dataRefined_fc128_innDrp0.1_k3.csv', 'fns_0.5', 'testing_model_dataRefined_fc128_innDrp0.1_k4.csv', 'fps_0.99', 'model_result_recording_fp0.99_fn0.5.csv', 'testing_model_dataRefined_fc128_innDrp0.1_k1.csv', 'testing_model_dataRefined_fc128_innDrp0.1_k5.csv', 'testing_model_dataRefined_fc128_innDrp0.1_k2.csv']


In [9]:
# merge all results files
result_files = glob.glob(dir_src + "testing*.csv")
print(result_files)
pd_all = pd.DataFrame()
for f in result_files:
    tmp = pd.read_csv(f)
    pd_all = pd.concat((pd_all, tmp))
print(pd_all.head())

['/home/gtx980/Desktop/CCP/Results/negative_samples/0714_5fold/testing_model_dataRefined_fc128_innDrp0.1_k3.csv', '/home/gtx980/Desktop/CCP/Results/negative_samples/0714_5fold/testing_model_dataRefined_fc128_innDrp0.1_k4.csv', '/home/gtx980/Desktop/CCP/Results/negative_samples/0714_5fold/testing_model_dataRefined_fc128_innDrp0.1_k1.csv', '/home/gtx980/Desktop/CCP/Results/negative_samples/0714_5fold/testing_model_dataRefined_fc128_innDrp0.1_k5.csv', '/home/gtx980/Desktop/CCP/Results/negative_samples/0714_5fold/testing_model_dataRefined_fc128_innDrp0.1_k2.csv']
                                            png_name    y_pred  y_true
0  /home/gtx980/Desktop/CCP/Data/DataSet_3/Clean_...  0.000222     0.0
1  /home/gtx980/Desktop/CCP/Data/DataSet_3/Clean_...  0.815540     0.0
2  /home/gtx980/Desktop/CCP/Data/DataSet_3/Clean_...  0.000065     0.0
3  /home/gtx980/Desktop/CCP/Data/DataSet_3/Clean_...  0.000189     0.0
4  /home/gtx980/Desktop/CCP/Data/DataSet_3/Clean_...  0.000285     0.0


In [10]:
#pd_all['y_true'] == 0 
fps = list(pd_all[(pd_all.y_pred >= fp_crit) & (pd_all.y_true == 0)].png_name)
fns = list(pd_all[(pd_all.y_pred < fn_crit) & (pd_all.y_true == 1)].png_name)
print(len(fps))
print(len(fns))

820
50


In [11]:
## deal with fps
ix = 0
for fimg in fps:
    fname = os.path.basename(fimg)
    #oname = dir_fp + 'prob_' + str(fp_prob[ix]) + '_' + fname # if we need output prob
    oname = dir_fp + fname
    shutil.copyfile(fimg, oname)
    ix += 1

In [12]:
## deal with fns
ix = 0
for fimg in fns:
    fname = os.path.basename(fimg)
    #oname = dir_fn + 'prob_' + str(fn_prob[ix]) + '_' + fname
    oname = dir_fn + fname
    shutil.copyfile(fimg, oname)
    ix += 1

In [13]:
# output csv to record it
out_fname = dir_src + 'model_result_recording_fp' + str(fp_crit) + '_fn' + str(fn_crit) + '.csv'

pd_fn = pd.DataFrame({'png_name': fns,
                      'model_predict': np.zeros(len(fns))})
pd_fp = pd.DataFrame({'png_name': fps,
                      'model_predict': np.ones(len(fps))})
pd_out = pd.concat((pd_fn, pd_fp))
pd_out.reset_index(drop = True)
pd_out.to_csv(out_fname)